In [235]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
import xgboost
from libs import common

from importlib import reload

In [2]:
hist = pd.read_csv('./data/Historical_Data.csv', parse_dates=['Date'], index_col='Date')
test = pd.read_csv('./data/Evaluation_Data.csv', parse_dates=['Date'], index_col='Date')

comb = pd.concat([hist, test], axis='index')

In [3]:
comb.head()

,TS_ID,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,Target
Date,,,,,,,,,,,,,,,,,
2017-01-01,TS4,ABC0,XYZ1,4390.0,2683.0,0.0,3171.0,0.0,0.0,0.0,0.0,0.0,488.0,0.0,0.0,2683.0,5.0
2017-01-01,TS5,ABC0,XYZ1,4390.0,2683.0,0.0,3171.0,0.0,0.0,0.0,0.0,0.0,488.0,0.0,0.0,2683.0,5.0
2017-01-01,TS6,ABC2,XYZ7,4390.0,2683.0,0.0,3171.0,0.0,0.0,0.0,0.0,0.0,488.0,0.0,0.0,2683.0,28.0
2017-01-01,TS7,ABC2,XYZ7,4390.0,2683.0,0.0,3171.0,0.0,0.0,0.0,0.0,0.0,488.0,0.0,0.0,2683.0,134.0
2017-01-01,TS9,ABC2,XYZ6,4390.0,2683.0,0.0,3171.0,0.0,0.0,0.0,0.0,0.0,488.0,0.0,0.0,2683.0,37.0


Re-index each time series from `2017-01-01` to `2020-01-31`

In [78]:
date_range = pd.date_range('2017-01-01', '2020-01-31', freq='D')
gb = comb.groupby('TS_ID', group_keys=False)

df_list = []

for key, group in gb:
    df = group.reindex(date_range)
    df['TS_ID'] = key   
    df_list.append(df)
    
reindexed = pd.concat(df_list)

Pre-process Target column

* Fill NA
* Treat outliers - using zscore threshold [-5,5]

In [79]:
# Fillna
reindexed['Target'].fillna(0, inplace=True)

In [80]:
# Comput Zscore
reindexed['product_mean'] = reindexed.groupby('TS_ID')['Target'].transform(np.mean)
reindexed['product_std'] = reindexed.groupby('TS_ID')['Target'].transform(np.std, ddof=0)
reindexed['product_zscore'] = (reindexed['Target'] - reindexed['product_mean'])/reindexed['product_std']

In [84]:
# Treat outliers by `zscore clipping`
reindexed["outlier"] = (reindexed["product_zscore"]>5).astype(int)
reindexed['_target'] = np.where(reindexed['outlier']==1, np.nan, reindexed['Target'])
reindexed['_product_max'] = reindexed.groupby('TS_ID')['_target'].transform(max)
reindexed['_target_clipped']= reindexed['Target'].clip(lower=0)
reindexed['_target_clipped']= reindexed['Target'].clip(upper=reindexed['_product_max'])

In [126]:
reindexed['_target_clipped'].clip(lower=0, inplace=True)

Interpolate marketing spend on days with no sales

In [107]:
def interpolate(group_df):
    df = group_df[['V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16']].interpolate(method='time').bfill()
    return df

def ffill_bfill(group_df):
    df = group_df.ffill().bfill()
    return df

spending_interpolate = reindexed.groupby('TS_ID').apply(interpolate).bfill()
reindexed[['V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16']] = spending_interpolate

In [110]:
v2_v3_fill = reindexed.groupby('TS_ID')[['V2', 'V3']].apply(ffill_bfill)
reindexed[['V2', 'V3']] = v2_v3_fill

In [111]:
reindexed.isnull().sum()

TS_ID               0
V2                  0
V3                  0
V4                  0
V5                  0
V6                  0
V7                  0
V8                  0
V9                  0
V10                 0
V11                 0
V12                 0
V13                 0
V14                 0
V15                 0
V16                 0
Target              0
product_mean        0
product_std         0
product_zscore      0
outlier             0
_target            60
_product_max        0
_target_clipped     0
dtype: int64

In [112]:
reindexed.to_csv('./data/reindexed.csv', index=True)

In [121]:
reindexed.head()

,TS_ID,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,cos_dow,sin_woy,cos_woy,sin_doy,cos_doy,sin_moy,cos_moy,sin_qtr,cos_qtr,is_weekend
2017-01-01,TS0,ABC0,XYZ2,3750.0,0.0,3000.0,750.0,750.0,0.0,0.0,...,1.000000,0.0,1.0,0.017213,0.999852,0.5,0.866025,1.0,6.123234e-17,1
2017-01-02,TS0,ABC0,XYZ2,3750.0,0.0,3000.0,750.0,750.0,0.0,0.0,...,0.623490,0.120537,0.992709,0.034422,0.999407,0.5,0.866025,1.0,6.123234e-17,0
2017-01-03,TS0,ABC0,XYZ2,5639.0,2131.0,1607.0,787.0,328.0,1672.0,1016.0,...,-0.222521,0.120537,0.992709,0.051620,0.998667,0.5,0.866025,1.0,6.123234e-17,0
2017-01-04,TS0,ABC0,XYZ2,6332.0,1504.0,1425.0,158.0,317.0,1583.0,369.0,...,-0.900969,0.120537,0.992709,0.068802,0.997630,0.5,0.866025,1.0,6.123234e-17,0
2017-01-05,TS0,ABC0,XYZ2,6063.0,1365.0,1937.0,222.0,286.0,1079.0,698.0,...,-0.900969,0.120537,0.992709,0.085965,0.996298,0.5,0.866025,1.0,6.123234e-17,0


### Feature Engineering

In [113]:
reload(common)

<module 'libs.common' from 'c:\\Users\\9510700\\Documents\\Finning\\forecasting\\libs\\common.py'>

In [130]:
df = common.create_temporal_features(reindexed)
df = common.define_holidays(df)
df = common.holidays_in_N_days(df, 10)

In [115]:
df.head()

,TS_ID,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,sin_doy,cos_doy,sin_moy,cos_moy,sin_qtr,cos_qtr,is_weekend,is_nat_holiday,is_holiday,holidays_in_last_10_days
2017-01-01,TS0,ABC0,XYZ2,3750.0,0.0,3000.0,750.0,750.0,0.0,0.0,...,0.017213,0.999852,0.5,0.866025,1.0,6.123234e-17,1,0,1,NaN
2017-01-01,TS1,ABC0,XYZ2,5639.0,2131.0,1607.0,787.0,328.0,1672.0,1016.0,...,0.017213,0.999852,0.5,0.866025,1.0,6.123234e-17,1,0,1,1.0
2017-01-01,TS10,ABC2,XYZ6,3750.0,0.0,3000.0,750.0,750.0,0.0,0.0,...,0.017213,0.999852,0.5,0.866025,1.0,6.123234e-17,1,0,1,2.0
2017-01-01,TS11,ABC0,XYZ0,5639.0,2131.0,1607.0,787.0,328.0,1672.0,1016.0,...,0.017213,0.999852,0.5,0.866025,1.0,6.123234e-17,1,0,1,3.0
2017-01-01,TS12,ABC0,XYZ0,5639.0,2131.0,1607.0,787.0,328.0,1672.0,1016.0,...,0.017213,0.999852,0.5,0.866025,1.0,6.123234e-17,1,0,1,4.0


In [131]:
lagged = df.groupby('TS_ID').apply(common.create_lags, '_target_clipped', 7)
lagged = lagged.groupby('TS_ID').apply(common.create_lags, '_target_clipped', 14)
lagged = lagged.groupby('TS_ID').apply(common.create_lags, '_target_clipped', 30)

rolling_mean = lagged.groupby('TS_ID').apply(common.rolling_mean, '_target_clipped', 14)
rolling_mean = rolling_mean.groupby('TS_ID').apply(common.rolling_mean, '_target_clipped', 14)
rolling_mean = rolling_mean.groupby('TS_ID').apply(common.rolling_mean, '_target_clipped', 30)

In [134]:
mom = rolling_mean.groupby('TS_ID').apply(common.momentum, '_target_clipped', 10)
mom = mom.groupby('TS_ID').apply(common.momentum, '_target_clipped', 30)

In [137]:
macd = mom.groupby('TS_ID').apply(common.MACD, 'Target', 7, 14)

In [139]:
macd[macd['TS_ID']=='TS0'].head(25)

,TS_ID,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,is_holiday,holidays_in_last_10_days,t-7D,t-14D,t-30D,14_day_rolling_mean,30_day_rolling_mean,M10,M30,7_14_MACD
2017-01-01,TS0,ABC0,XYZ2,3750.000000,0.000000,3000.00,750.000000,750.000000,0.000000,0.00,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2017-01-02,TS0,ABC0,XYZ2,3750.000000,0.000000,3000.00,750.000000,750.000000,0.000000,0.00,...,1,21.0,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,14.583333
2017-01-03,TS0,ABC0,XYZ2,5639.000000,2131.000000,1607.00,787.000000,328.000000,1672.000000,1016.00,...,0,42.0,NaN,NaN,NaN,62.500000,62.500000,NaN,NaN,87.393056
2017-01-04,TS0,ABC0,XYZ2,6332.000000,1504.000000,1425.00,158.000000,317.000000,1583.000000,369.00,...,0,42.0,NaN,NaN,NaN,265.666667,265.666667,NaN,NaN,157.472940
2017-01-05,TS0,ABC0,XYZ2,6063.000000,1365.000000,1937.00,222.000000,286.000000,1079.000000,698.00,...,0,42.0,NaN,NaN,NaN,422.250000,422.250000,NaN,NaN,103.042433
2017-01-06,TS0,ABC0,XYZ2,5411.000000,2785.000000,1108.00,918.000000,918.000000,1234.000000,696.00,...,0,42.0,NaN,NaN,NaN,353.800000,353.800000,NaN,NaN,116.611190
2017-01-07,TS0,ABC0,XYZ2,7821.000000,1026.000000,128.00,1282.000000,0.000000,2179.000000,128.00,...,1,42.0,NaN,NaN,NaN,383.000000,383.000000,NaN,NaN,42.677175
2017-01-08,TS0,ABC0,XYZ2,6652.500000,1700.500000,812.00,670.500000,396.000000,1925.500000,724.00,...,1,63.0,0.0,NaN,NaN,328.285714,328.285714,NaN,NaN,10.347493
2017-01-09,TS0,ABC0,XYZ2,5484.000000,2375.000000,1496.00,59.000000,792.000000,1672.000000,1320.00,...,0,84.0,125.0,NaN,NaN,287.250000,287.250000,NaN,NaN,237.721616
2017-01-10,TS0,ABC0,XYZ2,6703.000000,1063.000000,1608.00,490.000000,736.000000,1335.000000,436.00,...,0,84.0,672.0,NaN,NaN,492.222222,492.222222,NaN,NaN,112.524076


### Google Trend Data

In [161]:
cat = pd.read_csv('./data/GT_caterpillar.csv')
cat.head()

,Month,score
0,2017-01,76
1,2017-02,77
2,2017-03,78
3,2017-04,77
4,2017-05,79


In [170]:
cat.rename({'GT_score':'GT_score_CAT'}, axis='columns', inplace=True)

In [163]:
cat['year_month'] = pd.to_datetime(cat['Month'], format='%Y-%m')
cat.set_index('year_month',inplace=True)

In [164]:
date_range = pd.date_range(start='2017-01-01', end='2020-01-31')
cat = cat.reindex(date_range)
cat = cat.ffill()

In [166]:
cat.drop('Month', axis='columns', inplace=True)
cat.head(3)

,GT_score
2017-01-01,76.0
2017-01-02,76.0
2017-01-03,76.0


In [171]:
df = pd.merge(macd, cat, left_index=True, right_index=True, how='left', sort=False)
df.head()

,TS_ID,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,holidays_in_last_10_days,t-7D,t-14D,t-30D,14_day_rolling_mean,30_day_rolling_mean,M10,M30,7_14_MACD,GT_score_CAT
2017-01-01,TS0,ABC0,XYZ2,3750.0,0.0,3000.0,750.0,750.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,76.0
2017-01-01,TS1,ABC0,XYZ2,5639.0,2131.0,1607.0,787.0,328.0,1672.0,1016.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,76.0
2017-01-01,TS10,ABC2,XYZ6,3750.0,0.0,3000.0,750.0,750.0,0.0,0.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,76.0
2017-01-01,TS11,ABC0,XYZ0,5639.0,2131.0,1607.0,787.0,328.0,1672.0,1016.0,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,76.0
2017-01-01,TS12,ABC0,XYZ0,5639.0,2131.0,1607.0,787.0,328.0,1672.0,1016.0,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,76.0


In [180]:
df.to_csv('./data/un_lagged_dataset.csv', index=True)

### Most Important - Framing time-series

* `planning-lag` - Generally forecasting is carried out N days (eg. 10 days) before the start of the next month. Therfore need to lag features by N days
* `forecast_lag` - Since `target sales` is not available in test-set (Duhhhh!), and we need to forecast for next 30 days, we need to lag all feature values by 30 days
* Therfore total lag = (`planning_lag` + `forecast_lag`) 10+30 = 40 days
* Marketing spend figures are not available on test-set. Can either `INTERPOLATE` this data, OR use `30 DAY LAGGED VALUES`. For this assignment we will `INTERPOLATE`.

In [ ]:
[]

In [172]:
df.columns

Index(['TS_ID', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'Target', 'product_mean',
       'product_std', 'product_zscore', 'outlier', '_target', '_product_max',
       '_target_clipped', 'dow', 'woy', 'doy', 'moy', 'qtr', 'sin_dow',
       'cos_dow', 'sin_woy', 'cos_woy', 'sin_doy', 'cos_doy', 'sin_moy',
       'cos_moy', 'sin_qtr', 'cos_qtr', 'is_weekend', 'is_nat_holiday',
       'is_holiday', 'holidays_in_last_10_days', 't-7D', 't-14D', 't-30D',
       '14_day_rolling_mean', '30_day_rolling_mean', 'M10', 'M30', '7_14_MACD',
       'GT_score_CAT'],
      dtype='object')

In [176]:
features_to_lag = ['V2', 'V3', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 't-7D', 't-14D', 't-30D',
       '14_day_rolling_mean', '30_day_rolling_mean', 'M10', 'M30',
       '7_14_MACD', 'GT_score_CAT']

In [186]:
lagged_df = df.groupby('TS_ID', as_index=True)[features_to_lag].shift(periods=40, freq='D')
lagged_df.head()


V4      V5      V6     V7     V8      V9     V10  \
TS_ID                                                                    
TS0   2017-02-10  3750.0     0.0  3000.0  750.0  750.0     0.0     0.0   
      2017-02-11  3750.0     0.0  3000.0  750.0  750.0     0.0     0.0   
      2017-02-12  5639.0  2131.0  1607.0  787.0  328.0  1672.0  1016.0   
      2017-02-13  6332.0  1504.0  1425.0  158.0  317.0  1583.0   369.0   
      2017-02-14  6063.0  1365.0  1937.0  222.0  286.0  1079.0   698.0   

                     V11    V12    V13  ...     V16  t-7D  t-14D  t-30D  \
TS_ID                                   ...                               
TS0   2017-02-10  2000.0    0.0    0.0  ...  3250.0   NaN    NaN    NaN   
      2017-02-11  2000.0    0.0    0.0  ...  3250.0   NaN    NaN    NaN   
      2017-02-12   754.0  131.0  262.0  ...  3574.0   NaN    NaN    NaN   
      2017-02-13   475.0  792.0  343.0  ...  4908.0   NaN    NaN    NaN   
      2017-02-14   444.0  159.0  222.0  ...  5619.0   NaN    NaN    NaN   

                  14_day_rolling_mean  30_day_rolling_mean  M10  M30  \
TS_ID                                                                  
TS0   2017-02-10                  NaN                  NaN  NaN  NaN   
      2017-02-11             0.000000             0.000000  NaN  NaN   
      2017-02-12            62.500000            62.500000  NaN  NaN   
      2017-02-13           265.666667           265.666667  NaN  NaN   
      2017-02-14           422.250000           422.250000  NaN  NaN   

                   7_14_MACD  GT_score_CAT  
TS_ID                                       
TS0   2017-02-10    0.000000          76.0  
      2017-02-11   14.583333          76.0  
      2017-02-12   87.393056          76.0  
      2017-02-13  157.472940          76.0  
      2017-02-14  103.042433          76.0  

[5 rows x 22 columns]

In [188]:
temp = df.set_index('TS_ID', append=True).swaplevel()
temp.head()

,,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,...,holidays_in_last_10_days,t-7D,t-14D,t-30D,14_day_rolling_mean,30_day_rolling_mean,M10,M30,7_14_MACD,GT_score_CAT
TS_ID,,,,,,,,,,,,,,,,,,,,,,
TS0,2017-01-01,ABC0,XYZ2,3750.0,0.0,3000.0,750.0,750.0,0.0,0.0,2000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,76.0
TS1,2017-01-01,ABC0,XYZ2,5639.0,2131.0,1607.0,787.0,328.0,1672.0,1016.0,754.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,76.0
TS10,2017-01-01,ABC2,XYZ6,3750.0,0.0,3000.0,750.0,750.0,0.0,0.0,2000.0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,76.0
TS11,2017-01-01,ABC0,XYZ0,5639.0,2131.0,1607.0,787.0,328.0,1672.0,1016.0,754.0,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,76.0
TS12,2017-01-01,ABC0,XYZ0,5639.0,2131.0,1607.0,787.0,328.0,1672.0,1016.0,754.0,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,76.0


In [189]:
left, right = lagged_df.align(temp, join='left', axis=0)

In [191]:
left, right = left.align(right, join='right', axis=1)

In [192]:
left

V2  V3      V4      V5      V6     V7     V8      V9  \
TS_ID                                                                    
TS0   2017-02-10 NaN NaN  3750.0     0.0  3000.0  750.0  750.0     0.0   
      2017-02-11 NaN NaN  3750.0     0.0  3000.0  750.0  750.0     0.0   
      2017-02-12 NaN NaN  5639.0  2131.0  1607.0  787.0  328.0  1672.0   
      2017-02-13 NaN NaN  6332.0  1504.0  1425.0  158.0  317.0  1583.0   
      2017-02-14 NaN NaN  6063.0  1365.0  1937.0  222.0  286.0  1079.0   
...               ..  ..     ...     ...     ...    ...    ...     ...   
TS9   2020-03-07 NaN NaN   454.0  6021.0   639.0  302.0   82.0   316.0   
      2020-03-08 NaN NaN   454.0  6021.0   639.0  302.0   82.0   316.0   
      2020-03-09 NaN NaN   454.0  6021.0   639.0  302.0   82.0   316.0   
      2020-03-10 NaN NaN   454.0  6021.0   639.0  302.0   82.0   316.0   
      2020-03-11 NaN NaN   454.0  6021.0   639.0  302.0   82.0   316.0   

                     V10     V11  ...  holidays_in_last_10_days  t-7D  t-14D  \
TS_ID                             ...                                          
TS0   2017-02-10     0.0  2000.0  ...                       NaN   NaN    NaN   
      2017-02-11     0.0  2000.0  ...                       NaN   NaN    NaN   
      2017-02-12  1016.0   754.0  ...                       NaN   NaN    NaN   
      2017-02-13   369.0   475.0  ...                       NaN   NaN    NaN   
      2017-02-14   698.0   444.0  ...                       NaN   NaN    NaN   
...                  ...     ...  ...                       ...   ...    ...   
TS9   2020-03-07  3402.0   110.0  ...                       NaN   0.0    0.0   
      2020-03-08  3402.0   110.0  ...                       NaN   0.0    0.0   
      2020-03-09  3402.0   110.0  ...                       NaN   0.0    0.0   
      2020-03-10  3402.0   110.0  ...                       NaN   0.0    0.0   
      2020-03-11  3402.0   110.0  ...                       NaN   0.0    0.0   

                  t-30D  14_day_rolling_mean  30_day_rolling_mean  M10    M30  \
TS_ID                                                                           
TS0   2017-02-10    NaN                  NaN                  NaN  NaN    NaN   
      2017-02-11    NaN             0.000000             0.000000  NaN    NaN   
      2017-02-12    NaN            62.500000            62.500000  NaN    NaN   
      2017-02-13    NaN           265.666667           265.666667  NaN    NaN   
      2017-02-14    NaN           422.250000           422.250000  NaN    NaN   
...                 ...                  ...                  ...  ...    ...   
TS9   2020-03-07   67.0             0.000000            18.300000  0.0  -67.0   
      2020-03-08    0.0             0.000000            16.066667  0.0    0.0   
      2020-03-09  227.0             0.000000            16.066667  0.0 -227.0   
      2020-03-10  255.0             0.000000             8.500000  0.0 -255.0   
      2020-03-11    0.0             0.000000             0.000000  0.0    0.0   

                   7_14_MACD  GT_score_CAT  
TS_ID                                       
TS0   2017-02-10    0.000000          76.0  
      2017-02-11   14.583333          76.0  
      2017-02-12   87.393056          76.0  
      2017-02-13  157.472940          76.0  
      2017-02-14  103.042433          76.0  
...                      ...           ...  
TS9   2020-03-07   -4.746716          83.0  
      2020-03-08   -4.121984          83.0  
      2020-03-09   -3.578508          83.0  
      2020-03-10   -3.105965          83.0  
      2020-03-11   -2.695280          83.0  

[23646 rows x 51 columns]

In [193]:
right

V2    V3           V4      V5           V6          V7  \
TS_ID                                                                        
TS0   2017-02-10  ABC0  XYZ2  5544.000000  1769.0  1531.000000  646.000000   
      2017-02-11  ABC0  XYZ2  5362.666667  2219.0  1235.333333  447.333333   
      2017-02-12  ABC0  XYZ2  5181.333333  2669.0   939.666667  248.666667   
      2017-02-13  ABC0  XYZ2  5000.000000  3119.0   644.000000   50.000000   
      2017-02-14  ABC0  XYZ2  5732.000000  1752.0  2229.000000  669.000000   
...                ...   ...          ...     ...          ...         ...   
TS9   2020-03-07   NaN   NaN          NaN     NaN          NaN         NaN   
      2020-03-08   NaN   NaN          NaN     NaN          NaN         NaN   
      2020-03-09   NaN   NaN          NaN     NaN          NaN         NaN   
      2020-03-10   NaN   NaN          NaN     NaN          NaN         NaN   
      2020-03-11   NaN   NaN          NaN     NaN          NaN         NaN   

                      V8      V9          V10         V11  ...  \
TS_ID                                                      ...   
TS0   2017-02-10  1395.0   578.0   884.000000  374.000000  ...   
      2017-02-11  1425.0  1062.0  1513.333333  315.333333  ...   
      2017-02-12  1455.0  1546.0  2142.666667  256.666667  ...   
      2017-02-13  1485.0  2030.0  2772.000000  198.000000  ...   
      2017-02-14   382.0  1847.0   350.000000  860.000000  ...   
...                  ...     ...          ...         ...  ...   
TS9   2020-03-07     NaN     NaN          NaN         NaN  ...   
      2020-03-08     NaN     NaN          NaN         NaN  ...   
      2020-03-09     NaN     NaN          NaN         NaN  ...   
      2020-03-10     NaN     NaN          NaN         NaN  ...   
      2020-03-11     NaN     NaN          NaN         NaN  ...   

                  holidays_in_last_10_days   t-7D   t-14D   t-30D  \
TS_ID                                                               
TS0   2017-02-10                      42.0  888.0   139.0   526.0   
      2017-02-11                      42.0  143.0     0.0  1129.0   
      2017-02-12                      63.0    0.0     0.0     0.0   
      2017-02-13                      84.0    0.0   126.0     0.0   
      2017-02-14                      84.0   58.0  1598.0     0.0   
...                                    ...    ...     ...     ...   
TS9   2020-03-07                       NaN    NaN     NaN     NaN   
      2020-03-08                       NaN    NaN     NaN     NaN   
      2020-03-09                       NaN    NaN     NaN     NaN   
      2020-03-10                       NaN    NaN     NaN     NaN   
      2020-03-11                       NaN    NaN     NaN     NaN   

                  14_day_rolling_mean  30_day_rolling_mean     M10     M30  \
TS_ID                                                                        
TS0   2017-02-10           312.928571           371.000000 -1282.0  -210.0   
      2017-02-11           350.000000           375.400000   628.0  -471.0   
      2017-02-12           350.000000           337.766667 -1083.0     0.0   
      2017-02-13           350.000000           337.766667  -888.0     0.0   
      2017-02-14           490.285714           407.433333  1947.0  2090.0   
...                               ...                  ...     ...     ...   
TS9   2020-03-07                  NaN                  NaN     NaN     NaN   
      2020-03-08                  NaN                  NaN     NaN     NaN   
      2020-03-09                  NaN                  NaN     NaN     NaN   
      2020-03-10                  NaN                  NaN     NaN     NaN   
      2020-03-11                  NaN                  NaN     NaN     NaN   

                   7_14_MACD  GT_score_CAT  
TS_ID                                       
TS0   2017-02-10  -12.916547          77.0  
      2017-02-11  -44.768786          77.0  
      2017-02-12  -63.980449          77.0  
      20

In [202]:
right.head()

V2    V3           V4      V5           V6          V7  \
TS_ID                                                                        
TS0   2017-02-10  ABC0  XYZ2  5544.000000  1769.0  1531.000000  646.000000   
      2017-02-11  ABC0  XYZ2  5362.666667  2219.0  1235.333333  447.333333   
      2017-02-12  ABC0  XYZ2  5181.333333  2669.0   939.666667  248.666667   
      2017-02-13  ABC0  XYZ2  5000.000000  3119.0   644.000000   50.000000   
      2017-02-14  ABC0  XYZ2  5732.000000  1752.0  2229.000000  669.000000   

                      V8      V9          V10         V11  ...  \
TS_ID                                                      ...   
TS0   2017-02-10  1395.0   578.0   884.000000  374.000000  ...   
      2017-02-11  1425.0  1062.0  1513.333333  315.333333  ...   
      2017-02-12  1455.0  1546.0  2142.666667  256.666667  ...   
      2017-02-13  1485.0  2030.0  2772.000000  198.000000  ...   
      2017-02-14   382.0  1847.0   350.000000  860.000000  ...   

                  holidays_in_last_10_days   t-7D   t-14D   t-30D  \
TS_ID                                                               
TS0   2017-02-10                      42.0  888.0   139.0   526.0   
      2017-02-11                      42.0  143.0     0.0  1129.0   
      2017-02-12                      63.0    0.0     0.0     0.0   
      2017-02-13                      84.0    0.0   126.0     0.0   
      2017-02-14                      84.0   58.0  1598.0     0.0   

                  14_day_rolling_mean  30_day_rolling_mean     M10     M30  \
TS_ID                                                                        
TS0   2017-02-10           312.928571           371.000000 -1282.0  -210.0   
      2017-02-11           350.000000           375.400000   628.0  -471.0   
      2017-02-12           350.000000           337.766667 -1083.0     0.0   
      2017-02-13           350.000000           337.766667  -888.0     0.0   
      2017-02-14           490.285714           407.433333  1947.0  2090.0   

                   7_14_MACD  GT_score_CAT  
TS_ID                                       
TS0   2017-02-10  -12.916547          77.0  
      2017-02-11  -44.768786          77.0  
      2017-02-12  -63.980449          77.0  
      2017-02-13  169.497985          77.0  
      2017-02-14  103.275701          77.0  

[5 rows x 51 columns]

In [210]:
non_lag_cols = (df.columns).difference(features_to_lag).tolist()
non_lag_cols.remove('TS_ID')
left[non_lag_cols] = right[non_lag_cols]

In [211]:
left

V2    V3      V4      V5      V6     V7     V8      V9  \
TS_ID                                                                        
TS0   2017-02-10  ABC0  XYZ2  3750.0     0.0  3000.0  750.0  750.0     0.0   
      2017-02-11  ABC0  XYZ2  3750.0     0.0  3000.0  750.0  750.0     0.0   
      2017-02-12  ABC0  XYZ2  5639.0  2131.0  1607.0  787.0  328.0  1672.0   
      2017-02-13  ABC0  XYZ2  6332.0  1504.0  1425.0  158.0  317.0  1583.0   
      2017-02-14  ABC0  XYZ2  6063.0  1365.0  1937.0  222.0  286.0  1079.0   
...                ...   ...     ...     ...     ...    ...    ...     ...   
TS9   2020-03-07   NaN   NaN   454.0  6021.0   639.0  302.0   82.0   316.0   
      2020-03-08   NaN   NaN   454.0  6021.0   639.0  302.0   82.0   316.0   
      2020-03-09   NaN   NaN   454.0  6021.0   639.0  302.0   82.0   316.0   
      2020-03-10   NaN   NaN   454.0  6021.0   639.0  302.0   82.0   316.0   
      2020-03-11   NaN   NaN   454.0  6021.0   639.0  302.0   82.0   316.0   

                     V10     V11  ...  holidays_in_last_10_days  t-7D  t-14D  \
TS_ID                             ...                                          
TS0   2017-02-10     0.0  2000.0  ...                      42.0   NaN    NaN   
      2017-02-11     0.0  2000.0  ...                      42.0   NaN    NaN   
      2017-02-12  1016.0   754.0  ...                      63.0   NaN    NaN   
      2017-02-13   369.0   475.0  ...                      84.0   NaN    NaN   
      2017-02-14   698.0   444.0  ...                      84.0   NaN    NaN   
...                  ...     ...  ...                       ...   ...    ...   
TS9   2020-03-07  3402.0   110.0  ...                       NaN   0.0    0.0   
      2020-03-08  3402.0   110.0  ...                       NaN   0.0    0.0   
      2020-03-09  3402.0   110.0  ...                       NaN   0.0    0.0   
      2020-03-10  3402.0   110.0  ...                       NaN   0.0    0.0   
      2020-03-11  3402.0   110.0  ...                       NaN   0.0    0.0   

                  t-30D  14_day_rolling_mean  30_day_rolling_mean  M10    M30  \
TS_ID                                                                           
TS0   2017-02-10    NaN                  NaN                  NaN  NaN    NaN   
      2017-02-11    NaN             0.000000             0.000000  NaN    NaN   
      2017-02-12    NaN            62.500000            62.500000  NaN    NaN   
      2017-02-13    NaN           265.666667           265.666667  NaN    NaN   
      2017-02-14    NaN           422.250000           422.250000  NaN    NaN   
...                 ...                  ...                  ...  ...    ...   
TS9   2020-03-07   67.0             0.000000            18.300000  0.0  -67.0   
      2020-03-08    0.0             0.000000            16.066667  0.0    0.0   
      2020-03-09  227.0             0.000000            16.066667  0.0 -227.0   
      2020-03-10  255.0             0.000000             8.500000  0.0 -255.0   
      2020-03-11    0.0             0.000000             0.000000  0.0    0.0   

                   7_14_MACD  GT_score_CAT  
TS_ID                                       
TS0   2017-02-10    0.000000          76.0  
      2017-02-11   14.583333          76.0  
      2017-02-12   87.393056          76.0  
      2017-02-13  157.472940          76.0  
      2017-02-14  103.042433          76.0  
...                      ...           ...  
TS9   2020-03-07   -4.746716          83.0  
      2020-03-08   -4.121984          83.0  
      2020-03-09   -3.578508          83.0  
      2020-03-10   -3.105965          83.0  
      2020-03-11   -2.695280          83.0  

[23646 rows x 51 columns]

In [213]:
left = left.reset_index()
left.rename({'level_1': 'date'}, axis='columns', inplace=True)

In [214]:
left.head()

,TS_ID,date,V2,V3,V4,V5,V6,V7,V8,V9,...,holidays_in_last_10_days,t-7D,t-14D,t-30D,14_day_rolling_mean,30_day_rolling_mean,M10,M30,7_14_MACD,GT_score_CAT
0,TS0,2017-02-10,ABC0,XYZ2,3750.0,0.0,3000.0,750.0,750.0,0.0,...,42.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,76.0
1,TS0,2017-02-11,ABC0,XYZ2,3750.0,0.0,3000.0,750.0,750.0,0.0,...,42.0,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,14.583333,76.0
2,TS0,2017-02-12,ABC0,XYZ2,5639.0,2131.0,1607.0,787.0,328.0,1672.0,...,63.0,NaN,NaN,NaN,62.500000,62.500000,NaN,NaN,87.393056,76.0
3,TS0,2017-02-13,ABC0,XYZ2,6332.0,1504.0,1425.0,158.0,317.0,1583.0,...,84.0,NaN,NaN,NaN,265.666667,265.666667,NaN,NaN,157.472940,76.0
4,TS0,2017-02-14,ABC0,XYZ2,6063.0,1365.0,1937.0,222.0,286.0,1079.0,...,84.0,NaN,NaN,NaN,422.250000,422.250000,NaN,NaN,103.042433,76.0


In [215]:
left.to_csv('./data/dataset.csv', index=True)

### Label encode categorical variables

In [219]:
from sklearn.preprocessing import LabelEncoder

cat_cols = ['TS_ID', 'V2', 'V3']

for col in cat_cols:
    _col_name = f'le_{col}'
    left[_col_name] = LabelEncoder().fit_transform(left[col])

In [226]:
left[['TS_ID', 'V2', 'V3', 'le_TS_ID', 'le_V2', 'le_V3']].tail(10)

,TS_ID,V2,V3,le_TS_ID,le_V2,le_V3
23636,TS9,NaN,NaN,20,3,8
23637,TS9,NaN,NaN,20,3,8
23638,TS9,NaN,NaN,20,3,8
23639,TS9,NaN,NaN,20,3,8
23640,TS9,NaN,NaN,20,3,8
23641,TS9,NaN,NaN,20,3,8
23642,TS9,NaN,NaN,20,3,8
23643,TS9,NaN,NaN,20,3,8
23644,TS9,NaN,NaN,20,3,8
23645,TS9,NaN,NaN,20,3,8


### Seprate train and test

In [229]:
left.set_index('date', inplace=True)

In [254]:
train = left.loc[:'2019-12-31', :]
test = left.loc['2020-01-01':'2020-01-31', :]

### Train XGBoost model

In [241]:
train['_target_clipped'].isnull().sum()

0

In [248]:
train_cols = ['le_TS_ID', 'le_V2', 'le_V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'sin_dow',
       'cos_dow', 'sin_doy', 'cos_doy', 'sin_moy',
       'cos_moy', 'sin_qtr', 'cos_qtr', 'is_holiday', 'holidays_in_last_10_days', 't-7D', 't-14D', 't-30D',
       '14_day_rolling_mean', '30_day_rolling_mean', 'M10', 'M30', '7_14_MACD',
       'GT_score_CAT']

In [249]:
model = xgboost.XGBRegressor(
    n_estimators=1000, 
    max_depth=7, 
    eta=0.1, 
    subsample=0.7,
    colsample_bytree=0.8)

model.fit(train[train_cols], train['_target_clipped'])

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, enable_categorical=False,
             eta=0.1, gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.100000001,
             max_delta_step=0, max_depth=7, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=None)

### Predict on Test Set

In [256]:
test.isnull().sum()

TS_ID                       0
V2                          0
V3                          0
V4                          0
V5                          0
V6                          0
V7                          0
V8                          0
V9                          0
V10                         0
V11                         0
V12                         0
V13                         0
V14                         0
V15                         0
V16                         0
Target                      0
product_mean                0
product_std                 0
product_zscore              0
outlier                     0
_target                     0
_product_max                0
_target_clipped             0
dow                         0
woy                         0
doy                         0
moy                         0
qtr                         0
sin_dow                     0
cos_dow                     0
sin_woy                     0
cos_woy                     0
sin_doy   

In [259]:
test['forecast'] = model.predict(test[train_cols])

In [260]:
test.head(50)

,TS_ID,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,14_day_rolling_mean,30_day_rolling_mean,M10,M30,7_14_MACD,GT_score_CAT,le_TS_ID,le_V2,le_V3,forecast
date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,TS0,ABC0,XYZ2,5457.000000,1677.000000,2500.0,213.000000,335.0,2012.000000,671.000000,...,290.285714,555.800000,578.0,450.0,105.897069,86.0,0,0,2,402.250793
2020-01-02,TS0,ABC0,XYZ2,7361.000000,833.000000,278.0,1250.000000,0.0,417.000000,0.000000,...,345.357143,538.300000,771.0,-280.0,18.051883,86.0,0,0,2,474.319427
2020-01-03,TS0,ABC0,XYZ2,1277.000000,3191.000000,851.0,3830.000000,0.0,426.000000,0.000000,...,345.357143,503.266667,-498.0,-1185.0,-43.499218,86.0,0,0,2,581.479980
2020-01-04,TS0,ABC0,XYZ2,4074.000000,2519.000000,2259.0,370.000000,556.0,1667.000000,741.000000,...,345.357143,463.766667,-160.0,0.0,61.209207,86.0,0,0,2,10.348746
2020-01-05,TS0,ABC0,XYZ2,5361.000000,1255.000000,2395.0,418.000000,532.0,951.000000,190.000000,...,428.285714,502.466667,1010.0,1161.0,70.062710,86.0,0,0,2,131.031143
2020-01-06,TS0,ABC0,XYZ2,3821.000000,1857.000000,3071.0,536.000000,107.0,1214.000000,429.000000,...,392.785714,524.833333,671.0,-226.0,272.398730,86.0,0,0,2,417.650696
2020-01-07,TS0,ABC0,XYZ2,4315.000000,1815.000000,3253.0,377.000000,274.0,1370.000000,205.000000,...,562.500000,574.133333,2189.0,1865.0,203.270518,86.0,0,0,2,1066.969360
2020-01-08,TS0,ABC0,XYZ2,4439.000000,1043.000000,3877.0,294.000000,214.0,1016.000000,134.000000,...,579.357143,581.566667,580.0,-1109.0,55.311497,86.0,0,0,2,519.425659
2020-01-09,TS0,ABC0,XYZ2,607.000000,6090.000000,137.0,326.000000,76.0,289.000000,3584.000000,...,567.928571,520.133333,0.0,0.0,-20.538917,86.0,0,0,2,363.935303
